# Random Forest — Customer Churn Prediction

## Step 1 — Load Data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../Data/Customer-Churn-Records.csv')
print('Shape:', df.shape)
df.head()


Shape: (10000, 18)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


## Step 2 — Data Cleaning

In [2]:
# Drop identifier columns with no predictive value
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

# Drop Complain — data leakage (directly causes churn, not a customer feature)
df.drop(['Complain'], axis=1, inplace=True)

# Remove duplicate rows
before = len(df)
df.drop_duplicates(inplace=True)
print(f'Duplicates removed: {before - len(df)}')
print(f'Remaining rows: {len(df)}')


Duplicates removed: 0
Remaining rows: 10000


In [3]:
# Check for missing values
print('Null values:\n', df.isnull().sum())
print('\nData types:\n', df.dtypes)


Null values:
 CreditScore           0
Geography             0
Gender                0
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
EstimatedSalary       0
Exited                0
Satisfaction Score    0
Card Type             0
Point Earned          0
dtype: int64

Data types:
 CreditScore             int64
Geography              object
Gender                 object
Age                     int64
Tenure                  int64
Balance               float64
NumOfProducts           int64
HasCrCard               int64
IsActiveMember          int64
EstimatedSalary       float64
Exited                  int64
Satisfaction Score      int64
Card Type              object
Point Earned            int64
dtype: object


## Step 3 — Data Preprocessing

In [4]:
# Check class distribution (target imbalance)
print('Target distribution:')
print(df['Exited'].value_counts())
print(f'\nChurn rate: {df["Exited"].mean():.2%}')


Target distribution:
Exited
0    7962
1    2038
Name: count, dtype: int64

Churn rate: 20.38%


In [5]:
# Check for outliers in numeric columns using IQR
numeric_cols = df.select_dtypes(include=['int64','float64']).drop(columns=['Exited']).columns
for col in numeric_cols:
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    print(f'{col}: {outliers} outliers')


CreditScore: 15 outliers
Age: 359 outliers
Tenure: 0 outliers
Balance: 0 outliers
NumOfProducts: 60 outliers
HasCrCard: 0 outliers
IsActiveMember: 0 outliers
EstimatedSalary: 0 outliers
Satisfaction Score: 0 outliers
Point Earned: 0 outliers


In [6]:
# Descriptive statistics
df.describe()


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Satisfaction Score,Point Earned
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203800,3.013800,606.515100
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402842,1.405919,225.924839
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,1.000000,119.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,2.000000,410.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,3.000000,605.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,4.000000,801.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,5.000000,1000.000000


## Step 4 — Feature Engineering

In [7]:
# Separate features and target
X = df.drop('Exited', axis=1)
y = df['Exited']
print('Features:', X.columns.tolist())
print('Target shape:', y.shape)


Features: ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Satisfaction Score', 'Card Type', 'Point Earned']
Target shape: (10000,)


## Step 5 — Feature Encoding

In [8]:
# One-hot encode categorical columns: Geography, Gender, Card Type
# drop_first=True avoids multicollinearity (dummy variable trap)
X = pd.get_dummies(X, columns=['Geography', 'Gender', 'Card Type'], drop_first=True)
print('Encoded feature columns:')
print(X.columns.tolist())
print('Shape:', X.shape)


Encoded feature columns:
['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Satisfaction Score', 'Point Earned', 'Geography_Germany', 'Geography_Spain', 'Gender_Male', 'Card Type_GOLD', 'Card Type_PLATINUM', 'Card Type_SILVER']
Shape: (10000, 16)


## Step 6 — Train / Test Split

In [9]:
from sklearn.model_selection import train_test_split

# stratify=y ensures both splits preserve the same churn ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'Train: {X_train.shape[0]} rows | Test: {X_test.shape[0]} rows')
print(f'Train churn rate: {y_train.mean():.2%} | Test churn rate: {y_test.mean():.2%}')


Train: 8000 rows | Test: 2000 rows
Train churn rate: 20.38% | Test churn rate: 20.40%


## Step 7 — Model Training

In [10]:
from sklearn.ensemble import RandomForestClassifier


In [11]:
# IMPROVED MODEL - Better regularization and more trees
model = RandomForestClassifier(
    # Number of trees
    n_estimators=200,              # INCREASED 200 
    
    # Tree complexity (reduce overfitting)
    max_depth=9,                   # REDUCED from 10
    min_samples_split=10,          # INCREASED from 5
    min_samples_leaf=4,            # INCREASED from 2
    
    # Feature sampling (improve generalization)
    max_features='sqrt',           # NEW: use sqrt(n_features)
    max_samples=0.8,               # NEW: bootstrap 80% samples
    
    # Class imbalance
    class_weight='balanced',
    
    # Performance
    random_state=42,
    n_jobs=-1,
    
    # Additional parameters
    bootstrap=True,
    oob_score=True                 # NEW: out-of-bag score
)

model.fit(X_train, y_train)
print(f'Random Forest trained')
print(f'OOB Score: {model.oob_score_:.4f}')


Random Forest trained
OOB Score: 0.8311


## Validation

In [12]:
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score,
    recall_score, confusion_matrix, classification_report
)
from sklearn.model_selection import cross_val_score
import numpy as np

train_preds = model.predict(X_train)
test_preds  = model.predict(X_test)

train_acc = accuracy_score(y_train, train_preds)
test_acc  = accuracy_score(y_test,  test_preds)
f1        = f1_score(y_test,        test_preds)
precision = precision_score(y_test, test_preds)
recall    = recall_score(y_test,    test_preds)

cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
cv_mean   = cv_scores.mean()
cv_std    = cv_scores.std()
gap       = train_acc - test_acc


In [13]:
# Train / Test Accuracy
print('=== Train / Test Accuracy ===')
print(f'Train Accuracy : {train_acc:.4f}')
print(f'Test  Accuracy : {test_acc:.4f}')


=== Train / Test Accuracy ===
Train Accuracy : 0.8752
Test  Accuracy : 0.8320


In [14]:
# Cross Validation Score
print('=== Cross Validation Score (5-fold) ===')
print(f'CV Mean  : {cv_mean:.4f}')
print(f'CV Std   : {cv_std:.4f}')
print(f'CV Scores: {cv_scores.round(4)}')


=== Cross Validation Score (5-fold) ===
CV Mean  : 0.8344
CV Std   : 0.0034
CV Scores: [0.828  0.837  0.8375 0.8345 0.835 ]


In [15]:
# Confusion Matrix
print('=== Confusion Matrix ===')
cm = confusion_matrix(y_test, test_preds)
print(cm)
tn, fp, fn, tp = cm.ravel()
print(f'True  Negatives (Correctly predicted Stay)  : {tn}')
print(f'False Positives (Said Churn, actually Stay) : {fp}')
print(f'False Negatives (Said Stay, actually Churn) : {fn}')
print(f'True  Positives (Correctly predicted Churn) : {tp}')


=== Confusion Matrix ===
[[1376  216]
 [ 120  288]]
True  Negatives (Correctly predicted Stay)  : 1376
False Positives (Said Churn, actually Stay) : 216
False Negatives (Said Stay, actually Churn) : 120
True  Positives (Correctly predicted Churn) : 288


In [16]:
# Precision, Recall, F1
print('=== Precision / Recall / F1 ===')
print(f'Precision : {precision:.4f}')
print(f'Recall    : {recall:.4f}')
print(f'F1 Score  : {f1:.4f}')
print()
print(classification_report(y_test, test_preds, target_names=['Stayed','Churned']))


=== Precision / Recall / F1 ===
Precision : 0.5714
Recall    : 0.7059
F1 Score  : 0.6316

              precision    recall  f1-score   support

      Stayed       0.92      0.86      0.89      1592
     Churned       0.57      0.71      0.63       408

    accuracy                           0.83      2000
   macro avg       0.75      0.79      0.76      2000
weighted avg       0.85      0.83      0.84      2000



In [17]:
# Overfitting Check
print('=== Overfitting Check ===')
print(f'Train Accuracy : {train_acc:.4f}')
print(f'Test  Accuracy : {test_acc:.4f}')
print(f'Gap            : {gap:.4f}')
if gap > 0.05:
    print('Warning: Overfitting detected (gap > 5%)')
else:
    print('Good: No significant overfitting (gap <= 5%)')


=== Overfitting Check ===
Train Accuracy : 0.8752
Test  Accuracy : 0.8320
Gap            : 0.0433
Good: No significant overfitting (gap <= 5%)


In [18]:
# Class Imbalance
print('=== Class Imbalance ===')
print(f'Stayed  (0): {(y==0).sum()} ({(y==0).mean()*100:.1f}%)')
print(f'Churned (1): {(y==1).sum()} ({(y==1).mean()*100:.1f}%)')
ratio = (y==0).sum() / (y==1).sum()
print(f'Imbalance ratio: {ratio:.2f}:1')
if ratio > 2:
    print('Dataset is imbalanced — class_weight=balanced is recommended')
else:
    print('Dataset is relatively balanced')


=== Class Imbalance ===
Stayed  (0): 7962 (79.6%)
Churned (1): 2038 (20.4%)
Imbalance ratio: 3.91:1
Dataset is imbalanced — class_weight=balanced is recommended


In [19]:
# Generalization Ability
print('=== Generalization Ability ===')
print(f'CV Mean Accuracy  : {cv_mean:.4f}')
print(f'Test Accuracy     : {test_acc:.4f}')
gen_gap = abs(cv_mean - test_acc)
print(f'CV vs Test gap    : {gen_gap:.4f}')
if gen_gap < 0.02:
    print('Excellent generalization — model performs consistently on unseen data')
elif gen_gap < 0.05:
    print('Good generalization — minor variance between CV and test')
else:
    print('Poor generalization — model is unstable across different data splits')


=== Generalization Ability ===
CV Mean Accuracy  : 0.8344
Test Accuracy     : 0.8320
CV vs Test gap    : 0.0024
Excellent generalization — model performs consistently on unseen data


In [20]:
# Feature Importance
import pandas as pd
imp = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
imp = imp.sort_values('Importance', ascending=False)
print('Top 10 Features:')
print(imp.head(10).to_string(index=False))


Top 10 Features:
          Feature  Importance
              Age    0.338463
    NumOfProducts    0.203929
          Balance    0.097983
Geography_Germany    0.053474
   IsActiveMember    0.053049
  EstimatedSalary    0.052766
     Point Earned    0.051973
      CreditScore    0.050635
           Tenure    0.027520
      Gender_Male    0.022523


In [21]:
# import mlflow, mlflow.sklearn
# mlflow.set_experiment('Customer_Churn_RandomForest_Improved')
# with mlflow.start_run():
#     # Log all parameters
#     mlflow.log_param('n_estimators',      200)
#     mlflow.log_param('max_depth',         9)
#     mlflow.log_param('min_samples_split', 10)
#     mlflow.log_param('min_samples_leaf',  4)
#     mlflow.log_param('max_features',      'sqrt')
#     mlflow.log_param('max_samples',       0.8)
#     mlflow.log_param('class_weight',      'balanced')
    
#     # Log metrics
#     mlflow.log_metric('train_accuracy',   train_acc)
#     mlflow.log_metric('test_accuracy',    test_acc)
#     mlflow.log_metric('f1_score',         f1)
#     mlflow.log_metric('precision',        precision)
#     mlflow.log_metric('recall',           recall)
#     mlflow.log_metric('cv_accuracy',      cv_mean)
#     mlflow.log_metric('overfitting_gap',  gap)
#     mlflow.log_metric('oob_score',        model.oob_score_)
    
#     mlflow.sklearn.log_model(model, name='random_forest_model_improved')
#     print('MLflow run logged')

In [22]:
import joblib
import json
import os

# Create Models directory
models_dir = '../Models'
os.makedirs(models_dir, exist_ok=True)

# 1. Save the model
model_path = os.path.join(models_dir, 'random_forest_model.pkl')
joblib.dump(model, model_path)

# 2. Save feature names (CRITICAL!)
feature_names_path = os.path.join(models_dir, 'feature_names.pkl')
joblib.dump(X.columns.tolist(), feature_names_path)

# 3. Save preprocessing info
preprocessing_info = {
    'categorical_columns': ['Geography', 'Gender', 'Card Type'],
    'numerical_columns': ['CreditScore', 'Age', 'Tenure', 'Balance', 
                         'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
                         'EstimatedSalary', 'Satisfaction Score', 'Point Earned'],
    'drop_first': True,
    'expected_features': X.columns.tolist(),
    'n_features': len(X.columns)
}
preprocessing_path = os.path.join(models_dir, 'preprocessing_info.pkl')
joblib.dump(preprocessing_info, preprocessing_path)

# 4. Save model metadata
metadata = {
    'model_type': 'RandomForestClassifier',
    'test_accuracy': float(test_acc),
    'f1_score': float(f1),
    'precision': float(precision),
    'recall': float(recall),
    'n_features': len(X.columns),
    'feature_names': X.columns.tolist()
}
metadata_path = os.path.join(models_dir, 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4)

print("=" * 70)
print("✅ MODEL PACKAGE SAVED SUCCESSFULLY!")
print("=" * 70)
print(f"📦 Model:           {model_path}")
print(f"📋 Features:        {feature_names_path}")
print(f"⚙️  Preprocessing:   {preprocessing_path}")
print(f"📄 Metadata:        {metadata_path}")
print(f"\n🎯 Test Accuracy:   {test_acc:.4f}")
print(f"📊 F1 Score:        {f1:.4f}")
print(f"🔢 Total Features:  {len(X.columns)}")
print("=" * 70)

✅ MODEL PACKAGE SAVED SUCCESSFULLY!
📦 Model:           ../Models\random_forest_model.pkl
📋 Features:        ../Models\feature_names.pkl
⚙️  Preprocessing:   ../Models\preprocessing_info.pkl
📄 Metadata:        ../Models\model_metadata.json

🎯 Test Accuracy:   0.8320
📊 F1 Score:        0.6316
🔢 Total Features:  16
